In [ ]:
# Importing the necessary libraries
 
import numpy as np
import pickle
import gzip
import random
import sklearn as sk
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score  
import pydotplus
import matplotlib.pyplot as plt
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from keras.models import Sequential
from keras.layers import Dense
 
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
 
import keras
import glob
import cv2 as cv
import matplotlib.pyplot as plt 
import matplotlib.image as img 
from PIL import Image
import os
import zipfile
import tensorflow as tf
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
# Mount the google drive for the input dataset

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Unzip the input dataset. Run this only once

#zip_file = zipfile.ZipFile('/content/drive/My Drive/AR_FaceDB.zip', 'r')
#zip_file.extractall('/content/drive/My Drive/')
#zip_file.close()

In [ ]:
# Defining the labels for the dataset as per the input dataset labels

labels = ['specs', 'nonSpecs']

In [ ]:
# Create the training dataframe for training by combining the specs and non-specs data

train_labels   = []
train_path     = []

for label in labels:
  training_path  = glob.glob('/content/drive/My Drive/AR_FaceDB/' + label + '_train/*.bmp')

  for path in training_path:
    train_path.append(path)
    train_labels.append(label)

train_dataframe = pd.DataFrame({'path' : train_path, 'label' : train_labels})
print(train_dataframe)

                                                  path     label
0    /content/drive/My Drive/AR_FaceDB/specs_train/...     specs
1    /content/drive/My Drive/AR_FaceDB/specs_train/...     specs
2    /content/drive/My Drive/AR_FaceDB/specs_train/...     specs
3    /content/drive/My Drive/AR_FaceDB/specs_train/...     specs
4    /content/drive/My Drive/AR_FaceDB/specs_train/...     specs
..                                                 ...       ...
701  /content/drive/My Drive/AR_FaceDB/nonSpecs_tra...  nonSpecs
702  /content/drive/My Drive/AR_FaceDB/nonSpecs_tra...  nonSpecs
703  /content/drive/My Drive/AR_FaceDB/nonSpecs_tra...  nonSpecs
704  /content/drive/My Drive/AR_FaceDB/nonSpecs_tra...  nonSpecs
705  /content/drive/My Drive/AR_FaceDB/nonSpecs_tra...  nonSpecs

[706 rows x 2 columns]


In [ ]:
# Create the test dataframe for testing by combining the specs and non-specs data

test_path   = []
test_labels = []

for label in labels:
  testing_path  = glob.glob('/content/drive/My Drive/AR_FaceDB/' + label + '_test/*.bmp')

  for path in testing_path:
    test_path.append(path)
    test_labels.append(label)

test_dataframe = pd.DataFrame({'path' : test_path, 'label' : test_labels})
print(test_dataframe)

                                                  path     label
0    /content/drive/My Drive/AR_FaceDB/specs_test/m...     specs
1    /content/drive/My Drive/AR_FaceDB/specs_test/m...     specs
2    /content/drive/My Drive/AR_FaceDB/specs_test/m...     specs
3    /content/drive/My Drive/AR_FaceDB/specs_test/m...     specs
4    /content/drive/My Drive/AR_FaceDB/specs_test/m...     specs
..                                                 ...       ...
329  /content/drive/My Drive/AR_FaceDB/nonSpecs_tes...  nonSpecs
330  /content/drive/My Drive/AR_FaceDB/nonSpecs_tes...  nonSpecs
331  /content/drive/My Drive/AR_FaceDB/nonSpecs_tes...  nonSpecs
332  /content/drive/My Drive/AR_FaceDB/nonSpecs_tes...  nonSpecs
333  /content/drive/My Drive/AR_FaceDB/nonSpecs_tes...  nonSpecs

[334 rows x 2 columns]


In [ ]:
train_dataframe['gender']      = train_dataframe['path'].str.split('/').str[-1].str.split('-').str[0]
train_dataframe['identity']    = train_dataframe['path'].str.split('/').str[-1].str.split('-').str[1]
train_dataframe['attribute']   = train_dataframe['path'].str.split('/').str[-1].str.split('-').str[-1].str.split('.').str[0]

train_dataframe

,path,label,gender,identity,attribute
0,/content/drive/My Drive/AR_FaceDB/specs_train/...,specs,m,001,10
1,/content/drive/My Drive/AR_FaceDB/specs_train/...,specs,m,001,21
2,/content/drive/My Drive/AR_FaceDB/specs_train/...,specs,m,001,22
3,/content/drive/My Drive/AR_FaceDB/specs_train/...,specs,m,001,23
4,/content/drive/My Drive/AR_FaceDB/specs_train/...,specs,m,001,8
...,...,...,...,...,...
701,/content/drive/My Drive/AR_FaceDB/nonSpecs_tra...,nonSpecs,w,057,24
702,/content/drive/My Drive/AR_FaceDB/nonSpecs_tra...,nonSpecs,w,059,2
703,/content/drive/My Drive/AR_FaceDB/nonSpecs_tra...,nonSpecs,w,059,24
704,/content/drive/My Drive/AR_FaceDB/nonSpecs_tra...,nonSpecs,w,060,17


In [ ]:
test_dataframe['gender']      = test_dataframe['path'].str.split('/').str[-1].str.split('-').str[0]
test_dataframe['identity']    = test_dataframe['path'].str.split('/').str[-1].str.split('-').str[1]
test_dataframe['attribute']   = test_dataframe['path'].str.split('/').str[-1].str.split('-').str[-1].str.split('.').str[0]

test_dataframe

,path,label,gender,identity,attribute
0,/content/drive/My Drive/AR_FaceDB/specs_test/m...,specs,m,002,21
1,/content/drive/My Drive/AR_FaceDB/specs_test/m...,specs,m,003,11
2,/content/drive/My Drive/AR_FaceDB/specs_test/m...,specs,m,003,12
3,/content/drive/My Drive/AR_FaceDB/specs_test/m...,specs,m,003,13
4,/content/drive/My Drive/AR_FaceDB/specs_test/m...,specs,m,003,24
...,...,...,...,...,...
329,/content/drive/My Drive/AR_FaceDB/nonSpecs_tes...,nonSpecs,w,060,19
330,/content/drive/My Drive/AR_FaceDB/nonSpecs_tes...,nonSpecs,w,060,20
331,/content/drive/My Drive/AR_FaceDB/nonSpecs_tes...,nonSpecs,w,060,5
332,/content/drive/My Drive/AR_FaceDB/nonSpecs_tes...,nonSpecs,w,060,6


In [ ]:
train_dataframe['gender'].value_counts(normalize=True) * 100

m    84.135977
w    15.864023
Name: gender, dtype: float64

In [ ]:
test_dataframe['gender'].value_counts(normalize=True) * 100

m    72.754491
w    27.245509
Name: gender, dtype: float64

In [ ]:
# Split the training sets into 2 sets - one for training, the other for validation.
# Use the ImageDataGenerator to generate the images for training and validation dataframes

datagen         = ImageDataGenerator(rescale = 1./255., validation_split = 0.25)

train_generator = datagen.flow_from_dataframe(dataframe = train_dataframe, x_col = 'path', y_col = 'label', subset = 'training',
                                              batch_size = 32, seed = 42, shuffle=True, class_mode = 'binary', target_size = (32, 32))

valid_generator = datagen.flow_from_dataframe(dataframe = train_dataframe, x_col = 'path', y_col = 'label', subset = 'validation',
                                              batch_size = 32, seed = 42, shuffle=True, class_mode = 'binary', target_size = (32, 32))

Found 530 validated image filenames belonging to 2 classes.
Found 176 validated image filenames belonging to 2 classes.


In [ ]:
# Use the ImageDataGenerator to generate the images for test dataframes.

test_datagen   = ImageDataGenerator(rescale = 1./255.)

test_generator = test_datagen.flow_from_dataframe(dataframe = test_dataframe, x_col = 'path', y_col = 'label',
                                                  batch_size = 32, seed = 42, shuffle=False, class_mode = 'binary', target_size = (32, 32))

print(test_generator)

Found 334 validated image filenames belonging to 2 classes.


In [ ]:
# Build the model using the architecture specified

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation = 'relu', padding ='same', input_shape = (32, 32, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), padding = 'same'))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation = 'relu', padding ='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
model.add(Activation('relu'))

model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy',
                                                                          tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
# Fitting the model

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST  = test_generator.n//test_generator.batch_size

model.fit_generator(generator = train_generator, steps_per_epoch = STEP_SIZE_TRAIN, validation_data = valid_generator,
                    validation_steps = STEP_SIZE_VALID, epochs = 10)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
16/16 [==============================] - 460s 29s/step - loss: 0.2807 - accuracy: 0.5803 - precision: 0.4776 - recall: 0.1553 - val_loss: 0.2725 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
16/16 [==============================] - 2s 142ms/step - loss: 0.1361 - accuracy: 0.8153 - precision: 0.9214 - recall: 0.6143 - val_loss: 0.0049 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/10
16/16 [==============================] - 2s 147ms/step - loss: 0.0810 - accuracy: 0.8936 - precision: 0.8673 - recall: 0.8798 - val_loss: 0.0037 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/10
16/16 [==============================] - 2s 142ms/step - loss: 0.0630 - accuracy: 0.9157 - precision: 0.8977 - recall: 0.9061 - val_loss: 0.0124 - val_accuracy: 0.9750 - val_precision: 1.0000 - val_recall: 0.9750
Epoch 5/10
16/16

In [ ]:
# Evaluate the model on validation dataset

_, accuracy, precision, recall = model.evaluate_generator(generator = valid_generator, steps = STEP_SIZE_TEST)

print('Validation Dataset results:')
print('The accuracy is %.2f' % (accuracy * 100))
print('The precision is %.2f' % precision)
print('The recall is %.2f' % recall)

Instructions for updating:
Please use Model.evaluate, which supports generators.
Validation Dataset results:
The accuracy is 99.43
The precision is 1.00
The recall is 0.99


In [ ]:
# Evaluate the model on training dataset

_, accuracy, precision, recall = model.evaluate_generator(generator = train_generator, steps = STEP_SIZE_TEST)

print('Training Dataset results:')
print('The accuracy is %.2f' % (accuracy * 100))
print('The precision is %.2f' % precision)
print('The recall is %.2f' % recall)

Training Dataset results:
The accuracy is 95.31
The precision is 0.90
The recall is 1.00


In [ ]:
# Evaluate the model on test dataset

_, accuracy, precision, recall = model.evaluate_generator(generator = test_generator, steps = STEP_SIZE_TRAIN)

print('Test Dataset results:')
print('The accuracy is %.2f' % (accuracy * 100))
print('The precision is %.2f' % precision)
print('The recall is %.2f' % recall)

Test Dataset results:
The accuracy is 26.95
The precision is 0.36
The recall is 0.40


In [ ]:
test_generator.class_indices

{'nonSpecs': 0, 'specs': 1}

In [ ]:
# From the above predictions, we can clearly see a bias as only bright images are being marked as people with non-specs, while
# people wearing muffler and specs together are being marked with specs.
# Additionally, no woman has been marked as wearing specs in the category.

# This shows that the current dataset is not a reflection of the real world and hence, a classic example of bias.
# To remove this bias, lets add more data to our test dataset by adding data from the data folder.

# Create the data dataframe for testing by reading the data from the data folder

data_path   = glob.glob('/content/drive/My Drive/AR_FaceDB/data/' + '*.bmp')

data_dataframe = pd.DataFrame({'path' : data_path})
data_dataframe

,path
0,/content/drive/My Drive/AR_FaceDB/data/w-037-1...
1,/content/drive/My Drive/AR_FaceDB/data/w-037-1...
2,/content/drive/My Drive/AR_FaceDB/data/w-037-1...
3,/content/drive/My Drive/AR_FaceDB/data/w-037-1...
4,/content/drive/My Drive/AR_FaceDB/data/w-037-2...
...,...
2219,/content/drive/My Drive/AR_FaceDB/data/m-049-1...
2220,/content/drive/My Drive/AR_FaceDB/data/m-049-1...
2221,/content/drive/My Drive/AR_FaceDB/data/m-049-1...
2222,/content/drive/My Drive/AR_FaceDB/data/m-049-2...


In [ ]:
# Adding additional labels to the data dataframe for a better understanding of the data we are dealing with.

data_dataframe['gender']      = data_dataframe['path'].str.split('/').str[-1].str.split('-').str[0]
data_dataframe['identity']    = data_dataframe['path'].str.split('/').str[-1].str.split('-').str[1]
data_dataframe['attribute']   = data_dataframe['path'].str.split('/').str[-1].str.split('-').str[-1].str.split('.').str[0]

data_dataframe

,path,gender,identity,attribute
0,/content/drive/My Drive/AR_FaceDB/data/w-037-1...,w,037,15
1,/content/drive/My Drive/AR_FaceDB/data/w-037-1...,w,037,16
2,/content/drive/My Drive/AR_FaceDB/data/w-037-1...,w,037,17
3,/content/drive/My Drive/AR_FaceDB/data/w-037-1...,w,037,18
4,/content/drive/My Drive/AR_FaceDB/data/w-037-2...,w,037,2
...,...,...,...,...
2219,/content/drive/My Drive/AR_FaceDB/data/m-049-1...,m,049,15
2220,/content/drive/My Drive/AR_FaceDB/data/m-049-1...,m,049,18
2221,/content/drive/My Drive/AR_FaceDB/data/m-049-1...,m,049,19
2222,/content/drive/My Drive/AR_FaceDB/data/m-049-2...,m,049,2


In [ ]:
# Printing the images from the data dataframe.

data_path  = glob.glob('/content/drive/My Drive/AR_FaceDB/data/' + '*.bmp')

test1_datagen = ImageDataGenerator(rescale = 1./255.)

for path in data_path:
  new_image        = tf.keras.preprocessing.image.load_img(path, target_size = (32, 32))
  data_image_array = np.expand_dims(keras.preprocessing.image.img_to_array(new_image), axis = 0) / 255
  actual_image     = tf.keras.preprocessing.image.load_img(path)
  plt.imshow(actual_image)
  plt.show()

In [ ]:
# We have analysed this data and to improve the bias, or rather reduce it, we will now add specific data from this data folder to the trainining dataframe.

# Create a dataframe for training by adding the specs data

set1   = []

set1.append('/content/drive/My Drive/AR_FaceDB/data/m-049-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/m-055-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/m-058-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/m-061-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/m-063-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/m-064-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/m-070-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/m-071-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/m-072-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/m-075-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/w-001-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/w-004-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/w-012-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/w-021-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/w-022-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/w-027-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/w-029-1.bmp')
set1.append('/content/drive/My Drive/AR_FaceDB/data/w-058-1.bmp')

dataframe1 = pd.DataFrame({'path' : set1, 'label' : 'specs'})

dataframe1['gender']      = dataframe1['path'].str.split('/').str[-1].str.split('-').str[0]
dataframe1['identity']    = dataframe1['path'].str.split('/').str[-1].str.split('-').str[1]
dataframe1['attribute']   = dataframe1['path'].str.split('/').str[-1].str.split('-').str[-1].str.split('.').str[0]

dataframe1

,path,label,gender,identity,attribute
0,/content/drive/My Drive/AR_FaceDB/data/m-049-1...,specs,m,049,1
1,/content/drive/My Drive/AR_FaceDB/data/m-055-1...,specs,m,055,1
2,/content/drive/My Drive/AR_FaceDB/data/m-058-1...,specs,m,058,1
3,/content/drive/My Drive/AR_FaceDB/data/m-061-1...,specs,m,061,1
4,/content/drive/My Drive/AR_FaceDB/data/m-063-1...,specs,m,063,1
5,/content/drive/My Drive/AR_FaceDB/data/m-064-1...,specs,m,064,1
6,/content/drive/My Drive/AR_FaceDB/data/m-070-1...,specs,m,070,1
7,/content/drive/My Drive/AR_FaceDB/data/m-071-1...,specs,m,071,1
8,/content/drive/My Drive/AR_FaceDB/data/m-072-1...,specs,m,072,1
9,/content/drive/My Drive/AR_FaceDB/data/m-075-1...,specs,m,075,1


In [ ]:
train_dataframe.shape

(706, 5)

In [ ]:
# Concatenate the newly created dataframes above with the original training dataframe to create a new training dataframe.

train_dataframe1 = pd.concat([train_dataframe, dataframe1])

train_dataframe1

train_dataframe1.shape

(1295, 5)

In [ ]:
# Split the training sets into 2 sets - one for training, the other for validation.
# Use the ImageDataGenerator to generate the images for training and validation dataframes

datagen         = ImageDataGenerator(rescale = 1./255., validation_split = 0.25)

new_train_generator = datagen.flow_from_dataframe(dataframe = train_dataframe1, x_col = 'path', y_col = 'label', subset = 'training',
                                              batch_size = 32, seed = 42, shuffle=True, class_mode = 'binary', target_size = (32, 32))

valid_generator = datagen.flow_from_dataframe(dataframe = train_dataframe1, x_col = 'path', y_col = 'label', subset = 'validation',
                                              batch_size = 32, seed = 42, shuffle=True, class_mode = 'binary', target_size = (32, 32))

Found 972 validated image filenames belonging to 2 classes.
Found 323 validated image filenames belonging to 2 classes.


In [ ]:
# Build the model using the architecture specified

model1 = Sequential()

model1.add(Conv2D(32, kernel_size=(3, 3), padding ='same', input_shape = (32, 32, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D((2, 2), padding = 'same'))
model1.add(Dropout(0.45))

model1.add(Conv2D(64, (3, 3), padding ='same'))
model1.add(Activation('relu'))
model1.add(Dropout(0.45))

model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size = (2, 2)))
model1.add(Dropout(0.45))

model1.add(Flatten())

model1.add(Dense(128))
model1.add(Activation('relu'))
model1.add(Dropout(0.45))

model1.add(Dense(1, activation = 'sigmoid'))

model1.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy',
                                                                          tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
# Fitting the model

STEP_SIZE_TRAIN = train_generator.n//new_train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST  = test_generator.n//test_generator.batch_size

model1.fit_generator(generator = new_train_generator, steps_per_epoch = STEP_SIZE_TRAIN, validation_data = valid_generator,
                    validation_steps = STEP_SIZE_VALID, epochs = 10)

Epoch 1/10
16/16 [==============================] - 20s 1s/step - loss: 0.0956 - accuracy: 0.8906 - precision_1: 0.0625 - recall_1: 0.0238 - val_loss: 1.0000 - val_accuracy: 0.0000e+00 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/10
16/16 [==============================] - 5s 288ms/step - loss: 0.0894 - accuracy: 0.9106 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 1.0000 - val_accuracy: 0.0000e+00 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/10
16/16 [==============================] - 4s 232ms/step - loss: 0.1074 - accuracy: 0.8926 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 1.0000 - val_accuracy: 0.0000e+00 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 4/10
16/16 [==============================] - 3s 180ms/step - loss: 0.1118 - accuracy: 0.8882 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 1.0000 - val_accuracy: 0.0000e+00 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Ep

In [ ]:
# Evaluate the model on test dataset

_, accuracy, precision, recall = model1.evaluate_generator(generator = test_generator, steps = STEP_SIZE_TRAIN)

print('Test Dataset results:')
print('The accuracy is %.2f' % (accuracy * 100))
print('The precision is %.2f' % precision)
print('The recall is %.2f' % recall)  

Test Dataset results:
The accuracy is 44.31
The precision is 0.00
The recall is 0.00


In [ ]:
# Evaluate the model on training dataset

_, accuracy, precision, recall = model1.evaluate_generator(generator = new_train_generator, steps = STEP_SIZE_TEST)

print('Training Dataset results:')
print('The accuracy is %.2f' % (accuracy * 100))
print('The precision is %.2f' % precision)
print('The recall is %.2f' % recall)

Training Dataset results:
The accuracy is 92.81
The precision is 0.00
The recall is 0.00


In [ ]:
# Evaluate the model on validation dataset

_, accuracy, precision, recall = model.evaluate_generator(generator = valid_generator, steps = STEP_SIZE_TEST)

print('Validation Dataset results:')
print('The accuracy is %.2f' % (accuracy * 100))
print('The precision is %.2f' % precision)
print('The recall is %.2f' % recall)

Validation Dataset results:
The accuracy is 99.69
The precision is 1.00
The recall is 1.00
